# Vektorová a maticová matematika bez použitia Numpy

## Inicializácia

In [1]:
import math
import random

In [2]:
def randvector(n, start=-100, stop=100, *, dtype='f'):
    if start > stop:
        raise ValueError('Start value cannot be greater than stop value.')
    if dtype == 'f':
        return [start + random.random() * (stop - start) for i in range(n)]
    elif dtype == 'i':
        return [random.randint(int(start), int(stop)) for i in range(n)]
    elif dtype == 'c':
        return [complex(start + random.random() * (stop - start), start + random.random() * (stop - start)) for i in range(n)]
    else:
        return

----------
## Zopár konvencií

V tomto notebooku budeme používať bežné matematické výrazy ako *vektor*, *matica* apod. Vzhľadom na to, že dátové typy týchto názvov v čistom *Python*e neexistujú, dohodneme sa na použití nasledujúcich prezývok:

| Názov v tomto notebooku | Čo je tým myslené |
|:----------------------- | -----------------:|
| Vektor | Homogénny zoznam |
| Matica | Homogénny zoznam vektorov o jednotnej dĺžke |

-------------
## Vlastné matematické operácie

### 1. Násobenie vektorov

#### 1.1 Skalárne

Nech $a = (a_0, a_1,\ldots,a_{N-1})^{\top}$ a $b = (b_0, b_1,\ldots,b_{N-1})^{\top}$ sú vektory. Ich skalárnym súčinom (Ospravedlňujem sa za zanedbávanie všetkých matematických presností, skalárny súčin možno, samozrejme, definovať i inak než euklidovsky) nazveme výraz

$$ a \cdot b = \sum_{i=0}^{N-1} a_ib_i.$$

##### Na čo si treba pri implementácii dávať pozor

* Aby vektory skutočne boli vektormi (viď tabuľka vyššie)
* Aby mali vektory rovnaký počet prvkov

In [3]:
a = randvector(10, dtype='i')
b = randvector(10, dtype='i')

def ishomo(v):
    """Judges whether a list is homogeneous."""
    
    return all([type(element) is type(v[0]) for element in v])

def dot(a, b):
    """
    Vector dot product.
    
    Args:
        a (list): The first vector. Has to be homogeneous.
        b (list): The second vector. Has to be homogeneous.
        
    Returns:
        float or int: Dot product of `a` and `b`.
        
    Raises:
        ValueError: If dimensions mismatch is detected or any of the vectors is heterogeneous.
        
    """
    if len(a) != len(b): # Detecting length mismatch
        raise ValueError('Dimensions mismatch. Length of a ({:d}) != length of b ({:d})'.format(len(a), len(b)))
    elif (not ishomo(a)) or (not ishomo(b)): # Detecting heterogeneity
        raise ValueError('Input vectors are not instances of homogeneous list.')
    else:
        product = 0
        for i in range(len(a)):
            product += a[i] * b[i]      
        return product

In [4]:
print(dot(a, b))

-9592


#### 1.2 Vektorové

* Vektorový súčin bežne definujeme pre vektory v $\mathbb{R}^3$.
* $ a \times b = \lVert a\rVert \lVert b\rVert \sin(\angle) n_1$, kde $\angle$ je uhol, ktorý vektory $a$ a $b$ zvierajú a $n_1$ je jednotkový normálový vektor roviny, v ktorej ležia oba činitele. Smer vektoru $n$ je daný pravidlom pravej ruky.
* Praktický výpočet: definícia pomocou formálneho determinantu

$$a\times b=\left\lvert\begin{array}{ccc}i&j&k\\ a_1&a_2&a_3\\ b_1&b_2&b_3\end{array}\right\rvert$$

kde $i$, $j$, $k$ je štandardná báza $\mathbb{R}^3$.

##### Na čo si treba pri implementácii dávať pozor

* Homogenita vstupných polí
* Dátový typ ich prvkov (vektorový súčin definovaný nad reálnymi číslami)
* Počet prvkov vstupných vektorov (oba by mali byť trojprvkové)

In [5]:
def cross(a, b):
    """
    Vector cross product
    
    Args:
        a (list): The first vector. Has to be homogeneous.
        b (list): The second vector. Has to be homogeneous.
        
    Returns:
        list: Homogeneous list containing three values.
        
    """
    if (not ishomo(a)) or (not ishomo(b)): # Detecting heterogeneity
        raise ValueError('Input vectors are not instances of homogeneous list.')
    if not (len(a) == 3 and len(b) == 3): # Detecting length mismatch
        raise ValueError('Each of the input vectors has to contain exactly 3 elements.')
    else:
        return [a[1] * b[2] - a[2] * b[0],\
                a[2] * b[0] - a[0] * b[2],\
                a[0] * b[1] - a[1] * b[0]]

In [6]:
a = randvector(3, 0, 10, dtype='i')
b = randvector(3, 0, 10, dtype='i')

print(cross(a, b))

[7, 9, -18]


----------
### 2 Násobenie matíc

#### 2.1 Po prvkoch

* Rozmery všetkých matíc, vstupujúcich i vystupujúcich, musia byť rovnaké.

In [7]:
def mat_prod_ew(*matrices):
    """
    Element-wise matrix multiplication.
    
    Args:
        matrices (list of list each): Matrices to be multiplied, each one as a distinct argument
        
    Returns:
        list of list: Element-wise product of the matrices
        
    """
    n_matrices = len(matrices) # Number of matrices to multiply
    n_rows = [len(matrices[i]) for i in range(n_matrices)]
    if not all([n_rows[0] == n for n in n_rows]):
        raise ValueError('All the input matrices have to have the same number of rows. Row counts of the actual matrices are: {}'.format(n_rows))
    else:
        n_columns = []
        for matrix in matrices:
            for row in matrix:
                n_columns.append(len(row))
        if not all([n_columns[0] == n for n in n_columns]):
            raise ValueError('Each of the input matrices has to have each of its rows of the same length. The rowlength has to be the same for all the input matrices.')
        else:
            ewp = [[1] * n_columns[0] for r in range(n_rows[0])] # Initializing
            for r in range(n_rows[0]):
                for c in range(n_columns[0]):
                    for matrix in range(n_matrices):
                        ewp[r][c] *= matrices[matrix][r][c]
                        
            return ewp
        print(ewp)

In [8]:
m1 = [randvector(5, 0, 10, dtype='i') for i in range(6)]
m2 = [randvector(5, 0, 10, dtype='i') for i in range(6)]
m3 = [randvector(5, 0, 10, dtype='i') for i in range(6)]
mat_prod_ew(m1, m2, m3)

[[126, 560, 0, 240, 0],
 [108, 72, 180, 90, 630],
 [441, 0, 20, 72, 0],
 [0, 32, 15, 70, 600],
 [189, 108, 160, 160, 4],
 [0, 126, 216, 0, 640]]

#### 2.2 Maticovo

Majme matice

$$A = \left[ \begin{array}{cccc} A_{0,0}&A_{0,1}&\cdots &A_{0,c_A-1}\\ A_{1,0}&A_{1,1}&\cdots &A_{1,c_A-1}\\ \vdots&\vdots&\ddots&\vdots\\ A_{r_A-1,0}&A_{r_A-1,1}&\cdots &A_{r_A-1,c_A-1}\\  \end{array} \right]\hspace{2cm} B = \left[ \begin{array}{cccc} B_{0,0}&B_{0,1}&\cdots &B_{0,c_B-1}\\ B_{1,0}&B_{1,1}&\cdots &B_{1,c_B-1}\\ \vdots&\vdots&\ddots&\vdots\\ B_{r_B-1,0}&B_{r_B-1,1}&\cdots &B_{r_B-1,c_B-1}\\  \end{array} \right],$$

kde $r_A, c_A, r_B$ a $c_B$ sú počty riadkov a stĺpcov v maticiach $A$ a $B$.

Pokiaľ platí, že $c_A = r_B$, maticový súčin $A\cdot B$ definujeme ako maticu $P$ o rozmeroch $r_A\times c_B$, ktorej prvky sú

$$ P_{i,k} = \sum_{j=0}^{c_A - 1} A_{i,j}B_{j,k}.$$

##### Na čo si dať pri implementácii pozor 

* Rozmery vstupných matíc

In [9]:
def isfull(m):
    """Checks whether the given list of lists has the same number of elements in each of its sublists. """
    if all([type(element) is list for element in m]) and all([ishomo(element) for element in m]):
        if all([len(element) == len(m[0]) for element in m]):
            return True
        else:
            return False
    else:
        raise ValueError('Argument m has to be a homogeneous list of homogeneous lists.')

In [10]:
def mat_prod(A, B):
    """
    Matrix product A . B.
    
    Args:
        A (list of list): The first (left) matrix. Expected shape is i * j.
        B (list of list): The second (right) matrix. Expected shape is j * k.
        
    Returns:
        list of list: Matrix of shape i * k
        
    """
    if (not isfull(A)) or (not isfull(B)):
        raise ValueError('Both the A and the B arguments have to be homogeneous lists of homogeneous lists. For both of them, number of elements in each of their sublists has to be the same for all its sublists.')
    elif not (isinstance(A[0][0], int) or isinstance(A[0][0], float) or isinstance(A[0][0], complex)):
        raise ValueError('Elements of the input matrices have to be of one of {int, float, complex} types.')
    elif not (isinstance(B[0][0], int) or isinstance(B[0][0], float) or isinstance(B[0][0], complex)):
        raise ValueError('Elements of the input matrices have to be of one of {int, float, complex} types.')
    elif len(A[0]) != len(B):
        raise ValueError('Number of columns of the matrix A ({:d}) has to be equal to number of rows of the matrix B ({:d}).'.format(len(A[0])), len(B))
    else:
        product = [len(B[0]) * [0] for i in range(len(A))] # Initialization
        for i in range(len(A)):
            for k in range(len(B[0])):
                for j in range(len(B)):
                    product[i][k] += A[i][j] * B[j][k]
        return product

In [11]:
m1 = [randvector(3, 0, 10, dtype='i') for i in range(6)]
m2 = [randvector(6, 0, 10, dtype='i') for i in range(3)]

print(m2)
print(m1)

p = mat_prod(m2, m1)

print(p)

[[3, 5, 8, 2, 7, 6], [9, 7, 6, 9, 6, 5], [6, 1, 9, 6, 0, 8]]
[[10, 8, 9], [0, 3, 6], [2, 1, 9], [0, 7, 10], [0, 0, 4], [10, 1, 9]]
[[106, 67, 231], [152, 167, 336], [158, 110, 273]]


---------
##### Try - except - finally

* Ošetrovanie vstupov nie je nutné robiť iba pomocou podmienok (`if`)
* Pokiaľ vieme, že program, ktorému nebudú poskytnuté korektné vstupy skončí chybou, môžeme použiť vyhodnotenie pomocou `try` - `except` - `finally`:

```python
try:
    <Program, ktorý chceme uskutočniť>
except <Voliteľne: konkrétny názov výnimky>:
    <Program, ktorý bude uskutočnený v prípade chyby>
finally: # Nemusí byť prítomné, je to voliteľný príkaz
    <Program, ktorý bude uskutočnený v ktoromkoľvek prípade>
```

In [12]:
def mat_prod(A, B):
    """
    Matrix product A . B.
    
    Args:
        A (list of list): The first (left) matrix. Expected shape is i * j.
        B (list of list): The second (right) matrix. Expected shape is j * k.
        
    Returns:
        list of list: Matrix of shape i * k
        
    """
    try:
        product = [len(B[0]) * [0] for i in range(len(A))] # Initialization
        for i in range(len(A)):
            for k in range(len(B[0])):
                for j in range(len(B)):
                    product[i][k] += A[i][j] * B[j][k]
        return product
    except:
        print('Something got wrong.') # Use more explanatory statement.

In [13]:
m1 = [randvector(3, 0, 10, dtype='i') for i in range(6)]
m2 = [randvector(6, 0, 10, dtype='i') for i in range(3)]

print(m2)
print(m1)

p = mat_prod(m1, m1)

print(p)

p = mat_prod(m2, m1)

print(p)

[[10, 10, 7, 8, 9, 8], [10, 2, 0, 2, 10, 4], [4, 5, 3, 3, 7, 2]]
[[0, 9, 9], [3, 9, 5], [0, 9, 5], [2, 1, 6], [8, 7, 4], [10, 2, 10]]
Something got wrong.
None
[[198, 330, 339], [130, 188, 192], [97, 164, 142]]


-----
## A na čo to možno použiť?

* Cieľom bolo skôr naučiť sa používať výnimky (*Exceptions*) a ošetrovať program pred neželanými vstupmi
* Využitie sa, samozrejme, ponúka kdekoľvek, kde je treba násobiť vektory či matice, k dispozícii sú však i omnoho efektívnejšie implementácie
    * **Balíček itertools** ([dokumentácia](https://docs.python.org/3/library/itertools.html))
    * **Balíček Numpy** ([dokumentácia](https://numpy.org/doc/stable/)), ktorý je dnes na desktopoch pre vektorové operácie výrazne preferovaný
        * Niektoré jeho funkcie sú predkompilovaným kódom v C
        * Dosahuje niekoľkonásobne vyššie rýchlosti než použitie čistého *Python*u.